In [1]:
import numpy as np
import pandas as pd
import sys
sys.path.append("../fraud_detection/src/")
from util import s_to_time_format, string_to_datetime,hour_to_range
from tqdm import tqdm

def value_to_count(df_train, df_test, df_train_normal_cano_id, df_):
    # separate continuous feature and categorial features
    feats = ['acqic', 'bacno', 'cano', 'conam', 'contp', 'csmcu', 'ecfg', 'etymd',
       'flbmk', 'flg_3dsmk', 'hcefg', 'insfg', 'iterm', 'locdt',
       'mcc', 'mchno', 'ovrlt', 'scity', 'stocn', 'stscd', 'loctm_hour_of_day',
       'loctm_minute_of_hour', 'loctm_second_of_min'] 
    cont_feats = ['iterm', 
                  'locdt',
                  'loctm_hour_of_day',
                  'loctm_minute_of_hour', 
                  'loctm_second_of_min']
    feats = [f for f in feats if f not in cont_feats]
    # we only coner categorial features
    
    df = pd.concat([df_train[feats], df_test[feats]], axis = 0)
    for f in tqdm(feats):
        count_dict = df[f].value_counts(dropna = False).to_dict() 
        df_train_normal_cano_id[f] = df_train_normal_cano_id[f].apply(lambda v: count_dict[v])
        df_train[f] = df_train[f].apply(lambda v: count_dict[v])
        df_test[f] = df_test[f].apply(lambda v: count_dict[v])
        df_[f] = df_[f].apply(lambda v: count_dict[v])
    return df_train,df_test,df_train_normal_cano_id, df_

def feature_normalization_auto(df_train, df_test, df_train_normal_cano_id,df_):
    """
    return two inputs of autoencoder, one is for train and another one is for test
    """
    #from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler
    feats = ['acqic', 'bacno', 'cano', 'conam', 'contp', 'csmcu', 'ecfg', 'etymd',
       'flbmk', 'flg_3dsmk', 'hcefg', 'insfg', 'iterm', 'locdt',
       'mcc', 'mchno', 'ovrlt', 'scity', 'stocn', 'stscd', 'loctm_hour_of_day',
       'loctm_minute_of_hour', 'loctm_second_of_min']
    df = pd.concat([df_train[feats], df_test[feats]], axis = 0)


    for f in tqdm(feats):
        try:
            #scaler = MinMaxScaler()
            max_ = df[f].max()
            min_ = df[f].min()
            df_train_normal_cano_id[f] = df_train_normal_cano_id[f].apply(lambda x: (x-min_)/(max_-min_))
            df_[f] = df_[f].apply(lambda x: (x-min_)/(max_-min_))
            #df_test[f] = df_test[f].apply(lambda x: (x-min_)/(max_-min_))
        except:
            print(f)
    return df_train_normal_cano_id,df_

def partition_(df, num_features):
    data = []
    for i in range(len(df)):
        out = None
        if i == 0:
            out = np.concatenate(((np.zeros((2,num_features))),df.iloc[:1].values))
        elif i== 1:
            out = np.concatenate(((np.zeros((1,num_features))),df.iloc[:i+1].values))
        else:
            out = df.iloc[i+1-3:i+1].values
        data.append(out)
    return data

def partition(df_, sequence_length = 3):
    feats = [f for f in df_.columns if f not in {"fraud_ind","cano_help","locdt_help"}]
    sequences = []
    for _, df in df_.groupby(by = "cano_help"):
        data = partition_(df[feats], num_features = len(feats))
        for d in data:
            sequences.append(d)
    return sequences

df_train = pd.read_csv("/data/yunrui_li/fraud/dataset/train.csv")
df_test = pd.read_csv("/data/yunrui_li/fraud/dataset/test.csv")


for df in [df_train, df_test]:
    # pre-processing
    df["loctm_"] = df.loctm.astype(int).astype(str)
    df.loctm_ = df.loctm_.apply(s_to_time_format).apply(string_to_datetime)
    # time-related feature
    df["loctm_hour_of_day"] = df.loctm_.apply(lambda x: x.hour)
    df["loctm_minute_of_hour"] = df.loctm_.apply(lambda x: x.minute)
    df["loctm_second_of_min"] = df.loctm_.apply(lambda x: x.second)

    # removed the columns no need
    df.drop(columns = ["loctm_", "loctm","txkey"], axis = 1, inplace = True)

df_train["cano_locdt_index"] = ["{}_{}".format(str(i),str(j)) for i,j in zip(df_train.cano,df_train.locdt)]
df_test["cano_locdt_index"] = ["{}_{}".format(str(i),str(j)) for i,j in zip(df_test.cano,df_test.locdt)]

df_train["cano_help"] = df_train.cano
df_test["cano_help"] = df_test.cano

df_train["locdt_help"] = df_train.locdt
df_test["locdt_help"] = df_test.locdt

#-----------------------------
# feature extraction
#-----------------------------
df = pd.concat([df_train, df_test], axis = 0)
df.sort_values(by = ["cano", "locdt"], inplace = True)

#-----------------------------
# prepare training data
#-----------------------------
df_train.sort_values(by = ["cano", "locdt"], inplace = True)

# df_train, df_test = value_to_count(df_train, df_test)
# df_train, df_test = feature_normalization_auto(df_train, df_test)

fraud_cano_id = df_train[df_train.fraud_ind == 1].cano.unique().tolist()

df_train_normal_cano_id = df_train[~df_train.cano.isin(fraud_cano_id)]
print ("number of training data",df_train_normal_cano_id.shape)

df_train, df_test, df_train_normal_cano_id, df = value_to_count(df_train, df_test,df_train_normal_cano_id, df)
df_train_normal_cano_id, df = feature_normalization_auto(df_train, df_test,df_train_normal_cano_id, df)

#-----------------------------
# post-processing
#-----------------------------
df.drop(columns = ["fraud_ind"], axis = 1, inplace = True)
df_train_normal_cano_id.drop(columns = ["fraud_ind"], axis = 1, inplace = True)
feats = ['acqic', 'bacno', 'cano', 'conam', 'contp', 'csmcu', 'ecfg', 'etymd',
   'flbmk', 'flg_3dsmk', 'hcefg', 'insfg', 'iterm', 'locdt',
   'mcc', 'mchno', 'ovrlt', 'scity', 'stocn', 'stscd', 'loctm_hour_of_day',
   'loctm_minute_of_hour', 'loctm_second_of_min'] + ["cano_locdt_index","cano_help","locdt_help"]

df = df[feats]
df_train_normal_cano_id = df_train_normal_cano_id[feats]

/ldap_home/yunrui.li/.pyenv/versions/3.6.5/envs/deepts/lib/python3.6/site-packages/ipykernel_launcher.py:105: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



number of training data (1390382, 27)


  0%|          | 0/18 [00:00<?, ?it/s]/ldap_home/yunrui.li/.pyenv/versions/3.6.5/envs/deepts/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  0%|          | 0/23 [00:00<?, ?it/s]/ldap_home/yunrui.li/.pyenv/versions/3.6.5/envs/deepts/lib/python3.6/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 23/23 [00:51<00:00,  2.26s/it]
/ldap_home/yunrui.li/.pyenv/versions/3.6.5/envs/deepts/lib/python3.6/site-packages/p

In [2]:
len(fraud_cano_id)

8730

In [3]:
df_train_normal_cano_id.shape

(1390382, 26)

In [4]:
df_train_normal_cano_id

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,...,ovrlt,scity,stocn,stscd,loctm_hour_of_day,loctm_minute_of_hour,loctm_second_of_min,cano_locdt_index,cano_help,locdt_help
284164,0.245115,0.007168,0.007168,0.014266,1.0,1.000000,1.0,0.783407,1.000000,1.0,...,1.0,1.000000,1.000000,1.0,0.652174,0.322034,0.813559,0_1,0,1
1185304,0.245115,0.007168,0.007168,0.002860,1.0,1.000000,1.0,0.783407,1.000000,1.0,...,1.0,1.000000,1.000000,1.0,0.652174,0.745763,0.118644,0_4,0,4
726295,0.794559,0.007168,0.007168,0.047804,1.0,1.000000,1.0,0.783407,1.000000,1.0,...,1.0,0.118140,1.000000,1.0,0.608696,0.898305,0.711864,0_20,0,20
155960,0.245115,0.007168,0.007168,0.012381,1.0,1.000000,1.0,0.783407,1.000000,1.0,...,1.0,1.000000,1.000000,1.0,0.652174,0.372881,0.728814,0_29,0,29
418445,0.245115,0.007168,0.007168,0.003907,1.0,1.000000,1.0,0.783407,1.000000,1.0,...,1.0,1.000000,1.000000,1.0,0.608696,0.627119,0.169492,0_37,0,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
840043,0.001933,0.026882,0.009857,0.000018,1.0,0.028231,0.0,0.612633,1.000000,1.0,...,1.0,0.000281,0.025884,1.0,0.826087,0.711864,0.949153,213329_89,213329,89
1506705,0.032094,0.002688,0.002688,0.004620,1.0,0.010258,1.0,1.000000,0.005172,0.0,...,1.0,0.000190,1.000000,1.0,0.695652,0.610169,0.745763,213334_89,213334,89
1509473,0.032094,0.002688,0.002688,0.002902,1.0,0.010258,1.0,1.000000,0.005172,0.0,...,1.0,0.000190,1.000000,1.0,0.826087,0.779661,0.661017,213334_90,213334,90
1509510,0.365315,0.002688,0.002688,1.000000,1.0,0.010258,1.0,1.000000,0.005172,0.0,...,1.0,1.000000,1.000000,1.0,0.652174,0.779661,0.711864,213334_90,213334,90


In [5]:
len(df)

1943452

In [6]:
df.shape

(1943452, 26)

In [7]:
pd.options.display.max_columns = 100
df

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,mcc,mchno,ovrlt,scity,stocn,stscd,loctm_hour_of_day,loctm_minute_of_hour,loctm_second_of_min,cano_locdt_index,cano_help,locdt_help
284164,0.245115,0.007168,0.007168,0.014266,1.000000,1.000000,1.0,0.783407,1.0,1.0,1.0,1.0,0.0,0.000000,1.000000,0.077168,1.0,1.000000,1.0,1.0,0.652174,0.322034,0.813559,0_1,0,1
1185304,0.245115,0.007168,0.007168,0.002860,1.000000,1.000000,1.0,0.783407,1.0,1.0,1.0,1.0,0.0,0.025210,1.000000,0.077168,1.0,1.000000,1.0,1.0,0.652174,0.745763,0.118644,0_4,0,4
726295,0.794559,0.007168,0.007168,0.047804,1.000000,1.000000,1.0,0.783407,1.0,1.0,1.0,1.0,0.0,0.159664,0.069285,0.000110,1.0,0.118140,1.0,1.0,0.608696,0.898305,0.711864,0_20,0,20
155960,0.245115,0.007168,0.007168,0.012381,1.000000,1.000000,1.0,0.783407,1.0,1.0,1.0,1.0,0.0,0.235294,1.000000,0.077168,1.0,1.000000,1.0,1.0,0.652174,0.372881,0.728814,0_29,0,29
418445,0.245115,0.007168,0.007168,0.003907,1.000000,1.000000,1.0,0.783407,1.0,1.0,1.0,1.0,0.0,0.302521,1.000000,0.077168,1.0,1.000000,1.0,1.0,0.608696,0.627119,0.169492,0_37,0,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104903,1.000000,0.000000,0.000000,0.000000,0.053372,1.000000,0.0,0.955971,1.0,1.0,1.0,1.0,0.0,0.991597,0.328259,0.039895,0.0,1.000000,1.0,1.0,0.565217,0.949153,0.050847,213570_119,213570,119
52429,0.000333,0.001792,0.000896,1.000000,1.000000,0.142137,1.0,0.324602,1.0,1.0,1.0,1.0,0.0,0.991597,0.110069,0.010656,1.0,0.196151,1.0,1.0,0.913043,0.779661,0.050847,213571_119,213571,119
52430,1.000000,0.001792,0.000896,0.002093,1.000000,1.000000,1.0,1.000000,1.0,1.0,1.0,1.0,0.0,0.991597,1.000000,0.047051,1.0,0.026956,1.0,1.0,0.913043,0.779661,0.372881,213571_119,213571,119
399288,1.000000,0.000000,0.000000,0.000006,0.053372,1.000000,0.0,0.955971,1.0,1.0,1.0,1.0,0.0,1.000000,0.328259,0.039895,1.0,1.000000,1.0,1.0,0.608696,0.237288,0.135593,213572_120,213572,120


In [8]:
df[df.cano_locdt_index == "4_58"]

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,mcc,mchno,ovrlt,scity,stocn,stscd,loctm_hour_of_day,loctm_minute_of_hour,loctm_second_of_min,cano_locdt_index,cano_help,locdt_help
1420933,0.794559,0.0,0.0,0.00402,1.0,1.0,1.0,0.074284,0.0,1.0,1.0,1.0,0.0,0.478992,1.0,0.006388,1.0,0.11814,1.0,1.0,0.782609,0.135593,0.457627,4_58,4,58


In [9]:
df_train_normal_cano_id[df_train_normal_cano_id.cano_locdt_index == "4_58"]

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,hcefg,insfg,iterm,locdt,mcc,mchno,ovrlt,scity,stocn,stscd,loctm_hour_of_day,loctm_minute_of_hour,loctm_second_of_min,cano_locdt_index,cano_help,locdt_help
1420933,0.794559,0.0,0.0,0.00402,1.0,1.0,1.0,0.074284,0.0,1.0,1.0,1.0,0.0,0.478992,1.0,0.006388,1.0,0.11814,1.0,1.0,0.782609,0.135593,0.457627,4_58,4,58


In [10]:
def partition_(df, num_features):
    data = []
    for i in range(len(df)):
        out = None
        if i == 0:
            out = np.concatenate(((np.zeros((2,num_features))),df.iloc[:1].values))
        elif i== 1:
            out = np.concatenate(((np.zeros((1,num_features))),df.iloc[:i+1].values))
        else:
            out = df.iloc[i+1-3:i+1].values
        data.append(out)
    return data

def partition(df_, sequence_length = 3):
    feats = [f for f in df_.columns if f not in {"fraud_ind","cano_help","locdt_help"}]
    sequences = []
    for _, df in df_.groupby(by = "cano_help"):
        data = partition_(df[feats], num_features = len(feats))
        for d in data:
            sequences.append(d)
    return sequences

def get_sequence_dataframe(df):
    df_train_sequences = partition(df)
    df_train_sequences = np.concatenate(df_train_sequences)
    df_train_sequences = pd.DataFrame(df_train_sequences)
    return df_train_sequences

# df_train_sequences = partition(df_train_normal_cano_id)
# X_train = np.concatenate(df_train_sequences)
# X_train = pd.DataFrame(X_train)
X_train = get_sequence_dataframe(df_train_normal_cano_id)
Feature = get_sequence_dataframe(df)

In [11]:
X_train.shape

(4171146, 24)

In [12]:
Feature.shape

(5830356, 24)

In [13]:
Feature

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0.245115,0.00716846,0.00716846,0.0142658,1,1,1,0.783407,1,1,1,1,0,0,1,0.0771681,1,1,1,1,0.652174,0.322034,0.813559,0_1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0.245115,0.00716846,0.00716846,0.0142658,1,1,1,0.783407,1,1,1,1,0,0,1,0.0771681,1,1,1,1,0.652174,0.322034,0.813559,0_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5830351,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5830352,1,0,0,5.94654e-06,0.0533723,1,0,0.955971,1,1,1,1,0,1,0.328259,0.0398945,1,1,1,1,0.608696,0.237288,0.135593,213572_120
5830353,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5830354,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
# def partition(df_, sequence_length = 3):
#     feats = [f for f in df_.columns if f not in {"fraud_ind"}]
#     sequences = []
#     i = 0
#     for _, df in df_[feats].groupby(by = "cano_locdt_index"):
#         data = partition_(df[feats], num_features = len(feats))
#         for d in data:
#             sequences.append(d)
#         i += 1
#         if i > 10:
#             break
#     return sequences
# X_train = get_sequence_dataframe(df_train_normal_cano_id)

In [15]:
X_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0.245115,0.00716846,0.00716846,0.0142658,1,1,1,0.783407,1,1,1,1,0,0,1,0.0771681,1,1,1,1,0.652174,0.322034,0.813559,0_1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0.245115,0.00716846,0.00716846,0.0142658,1,1,1,0.783407,1,1,1,1,0,0,1,0.0771681,1,1,1,1,0.652174,0.322034,0.813559,0_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4171141,0.0320937,0.00268817,0.00268817,0.00290191,1,0.0102579,1,1,0.00517195,0,0.00903582,1,0,0.747899,1,0.000141247,1,0.000190161,1,1,0.826087,0.779661,0.661017,213334_90
4171142,0.365315,0.00268817,0.00268817,1,1,0.0102579,1,1,0.00517195,0,0.00903582,1,0,0.747899,0.568704,0.000674849,1,1,1,1,0.652174,0.779661,0.711864,213334_90
4171143,0.0320937,0.00268817,0.00268817,0.00290191,1,0.0102579,1,1,0.00517195,0,0.00903582,1,0,0.747899,1,0.000141247,1,0.000190161,1,1,0.826087,0.779661,0.661017,213334_90
4171144,0.365315,0.00268817,0.00268817,1,1,0.0102579,1,1,0.00517195,0,0.00903582,1,0,0.747899,0.568704,0.000674849,1,1,1,1,0.652174,0.779661,0.711864,213334_90


In [ ]:
X_train.to_csv("X_train.csv", index = False)
Feature.to_csv("Feature.csv", index = False)

In [117]:
import tensorflow as tf
import torch
from tensorflow.python.client import device_lib
from torch.autograd import Variable


class GPUWrapper:
    def __init__(self, gpu):
        self.gpu = gpu

    @property
    def tf_device(self):
        local_device_protos = device_lib.list_local_devices()
        gpus = [x.name for x in local_device_protos if x.device_type == 'GPU']
        return tf.device(gpus[self.gpu] if gpus else '/cpu:0')

    @property
    def torch_device(self):
        return torch.device(f'cuda:{self.gpu}' if torch.cuda.is_available() else 'cpu')

    def to_var(self, x, **kwargs):
        """PyTorch only: send Var to proper device."""
        x = x.to(self.torch_device)
        return Variable(x, **kwargs)

    def to_device(self, model):
        """PyTorch only: send Model to proper device."""
        model.to(self.torch_device)

In [118]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from tqdm import trange
import sys
sys.path.append("../DeepADoTS/src/algorithms/")
# from .algorithm_utils import Algorithm, PyTorchUtils
# from .autoencoder import AutoEncoderModule
#from lstm_enc_dec_axl import LSTMEDModule
import abc
import logging
import random

import numpy as np
import torch
import tensorflow as tf
from tensorflow.python.client import device_lib
from torch.autograd import Variable


class Algorithm(metaclass=abc.ABCMeta):
    def __init__(self, module_name, name, seed, details=False):
        self.logger = logging.getLogger(module_name)
        self.name = name
        self.seed = seed
        self.details = details
        self.prediction_details = {}

        if self.seed is not None:
            random.seed(seed)
            np.random.seed(seed)

    def __str__(self):
        return self.name

    @abc.abstractmethod
    def fit(self, X):
        """
        Train the algorithm on the given dataset
        """

    @abc.abstractmethod
    def predict(self, X):
        """
        :return anomaly score
        """


class PyTorchUtils(metaclass=abc.ABCMeta):
    def __init__(self, seed, gpu):
        self.gpu = gpu
        self.seed = seed
        if self.seed is not None:
            torch.manual_seed(self.seed)
            torch.cuda.manual_seed(self.seed)
        self.framework = 0

    @property
    def device(self):
        #return 'cuda:0'
        return torch.device(f'cuda:{self.gpu}' if torch.cuda.is_available() and self.gpu is not None else 'cpu')

    def to_var(self, t, **kwargs):
        # ToDo: check whether cuda Variable.
        t = t.to(self.device)
        return Variable(t, **kwargs)

    def to_device(self, model):
        model.to(self.device)


class TensorflowUtils(metaclass=abc.ABCMeta):
    def __init__(self, seed, gpu):
        self.gpu = gpu
        self.seed = seed
        if self.seed is not None:
            tf.set_random_seed(seed)
        self.framework = 1

    @property
    def device(self):
        local_device_protos = device_lib.list_local_devices()
        gpus = [x.name for x in local_device_protos if x.device_type == 'GPU']
        return tf.device(gpus[self.gpu] if gpus and self.gpu is not None else '/cpu:0')
    
import logging

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from scipy.stats import multivariate_normal
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from tqdm import trange


class AutoEncoder(Algorithm, PyTorchUtils):
    def __init__(self, name: str='AutoEncoder', num_epochs: int=10, batch_size: int=20, lr: float=1e-3,
                 hidden_size: int=5, sequence_length: int=30, train_gaussian_percentage: float=0.25,
                 seed: int=None, gpu: int=None, details=True):
        Algorithm.__init__(self, __name__, name, seed, details=details)
        PyTorchUtils.__init__(self, seed, gpu)
        self.num_epochs = num_epochs
        self.batch_size = batch_size
        self.lr = lr

        self.hidden_size = hidden_size
        self.sequence_length = sequence_length
        self.train_gaussian_percentage = train_gaussian_percentage

        self.aed = None
        self.mean, self.cov = None, None

    def fit(self, X: pd.DataFrame):
        X.interpolate(inplace=True)
        X.bfill(inplace=True)
        data = X.values
        sequences = [data[i:i + self.sequence_length] for i in range(data.shape[0] - self.sequence_length + 1)]
        indices = np.random.permutation(len(sequences))
        split_point = int(self.train_gaussian_percentage * len(sequences))
        train_loader = DataLoader(dataset=sequences, batch_size=self.batch_size, drop_last=True,
                                  sampler=SubsetRandomSampler(indices[:-split_point]), pin_memory=True)
        train_gaussian_loader = DataLoader(dataset=sequences, batch_size=self.batch_size, drop_last=True,
                                           sampler=SubsetRandomSampler(indices[-split_point:]), pin_memory=True)

        self.aed = AutoEncoderModule(X.shape[1], self.sequence_length, self.hidden_size, seed=self.seed, gpu=self.gpu)
        self.to_device(self.aed)  # .double()
        optimizer = torch.optim.Adam(self.aed.parameters(), lr=self.lr)

        self.aed.train()
        for epoch in trange(self.num_epochs):
            logging.debug(f'Epoch {epoch+1}/{self.num_epochs}.')
            for ts_batch in train_loader:
                output = self.aed(self.to_var(ts_batch))
                loss = nn.MSELoss(size_average=False)(output, self.to_var(ts_batch.float()))
                self.aed.zero_grad()
                loss.backward()
                optimizer.step()

        self.aed.eval()
        error_vectors = []
        for ts_batch in train_gaussian_loader:
            output = self.aed(self.to_var(ts_batch))
            error = nn.L1Loss(reduce=False)(output, self.to_var(ts_batch.float()))
            error_vectors += list(error.view(-1, X.shape[1]).data.cpu().numpy())

        self.mean = np.mean(error_vectors, axis=0)
        self.cov = np.cov(error_vectors, rowvar=False)

    def predict(self, X: pd.DataFrame) -> np.array:
        X.interpolate(inplace=True)
        X.bfill(inplace=True)
        data = X.values
        sequences = [data[i:i + self.sequence_length] for i in range(data.shape[0] - self.sequence_length + 1)]
        data_loader = DataLoader(dataset=sequences, batch_size=self.batch_size, shuffle=False, drop_last=False)

        self.aed.eval()
        mvnormal = multivariate_normal(self.mean, self.cov, allow_singular=True)
        scores = []
        outputs = []
        errors = []
        for idx, ts in enumerate(data_loader):
            output = self.aed(self.to_var(ts))
            error = nn.L1Loss(reduce=False)(output, self.to_var(ts.float()))
            score = -mvnormal.logpdf(error.view(-1, X.shape[1]).data.cpu().numpy())
            scores.append(score.reshape(ts.size(0), self.sequence_length))
            if self.details:
                outputs.append(output.data.numpy())
                errors.append(error.data.numpy())

        # stores seq_len-many scores per timestamp and averages them
        scores = np.concatenate(scores)
        lattice = np.full((self.sequence_length, X.shape[0]), np.nan)
        for i, score in enumerate(scores):
            lattice[i % self.sequence_length, i:i + self.sequence_length] = score
        scores = np.nanmean(lattice, axis=0)

        if self.details:
            outputs = np.concatenate(outputs)
            lattice = np.full((self.sequence_length, X.shape[0], X.shape[1]), np.nan)
            for i, output in enumerate(outputs):
                lattice[i % self.sequence_length, i:i + self.sequence_length, :] = output
            self.prediction_details.update({'reconstructions_mean': np.nanmean(lattice, axis=0).T})

            errors = np.concatenate(errors)
            lattice = np.full((self.sequence_length, X.shape[0], X.shape[1]), np.nan)
            for i, error in enumerate(errors):
                lattice[i % self.sequence_length, i:i + self.sequence_length, :] = error
            self.prediction_details.update({'errors_mean': np.nanmean(lattice, axis=0).T})

        return scores


class AutoEncoderModule(nn.Module, PyTorchUtils):
    def __init__(self, n_features: int, sequence_length: int, hidden_size: int, seed: int, gpu: int):
        # Each point is a flattened window and thus has as many features as sequence_length * features
        super().__init__()
        PyTorchUtils.__init__(self, seed, gpu)
        input_length = n_features * sequence_length

        # creates powers of two between eight and the next smaller power from the input_length
        dec_steps = 2 ** np.arange(max(np.ceil(np.log2(hidden_size)), 2), np.log2(input_length))[1:]
        dec_setup = np.concatenate([[hidden_size], dec_steps.repeat(2), [input_length]])
        enc_setup = dec_setup[::-1]

        layers = np.array([[nn.Linear(int(a), int(b)), nn.Tanh()] for a, b in enc_setup.reshape(-1, 2)]).flatten()[:-1]
        self._encoder = nn.Sequential(*layers)
        self.to_device(self._encoder)

        layers = np.array([[nn.Linear(int(a), int(b)), nn.Tanh()] for a, b in dec_setup.reshape(-1, 2)]).flatten()[:-1]
        self._decoder = nn.Sequential(*layers)
        self.to_device(self._decoder)

    def forward(self, ts_batch, return_latent: bool=False):
        flattened_sequence = ts_batch.view(ts_batch.size(0), -1)
        enc = self._encoder(flattened_sequence.float())
        dec = self._decoder(enc)
        reconstructed_sequence = dec.view(ts_batch.size())
        return (reconstructed_sequence, enc) if return_latent else reconstructed_sequence
    
import logging

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from scipy.stats import multivariate_normal
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from tqdm import trange

# from .algorithm_utils import Algorithm, PyTorchUtils


class LSTMED(Algorithm, PyTorchUtils):
    def __init__(self, name: str='LSTM-ED', num_epochs: int=10, batch_size: int=20, lr: float=1e-3,
                 hidden_size: int=5, sequence_length: int=30, train_gaussian_percentage: float=0.25,
                 n_layers: tuple=(1, 1), use_bias: tuple=(True, True), dropout: tuple=(0, 0),
                 seed: int=None, gpu: int = None, details=True):
        Algorithm.__init__(self, __name__, name, seed, details=details)
        PyTorchUtils.__init__(self, seed, gpu)
        self.num_epochs = num_epochs
        self.batch_size = batch_size
        self.lr = lr

        self.hidden_size = hidden_size
        self.sequence_length = sequence_length
        self.train_gaussian_percentage = train_gaussian_percentage

        self.n_layers = n_layers
        self.use_bias = use_bias
        self.dropout = dropout

        self.lstmed = None
        self.mean, self.cov = None, None

    def fit(self, X: pd.DataFrame):
        X.interpolate(inplace=True)
        X.bfill(inplace=True)
        data = X.values
        sequences = [data[i:i + self.sequence_length] for i in range(data.shape[0] - self.sequence_length + 1)]
        indices = np.random.permutation(len(sequences))
        split_point = int(self.train_gaussian_percentage * len(sequences))
        train_loader = DataLoader(dataset=sequences, batch_size=self.batch_size, drop_last=True,
                                  sampler=SubsetRandomSampler(indices[:-split_point]), pin_memory=True)
        train_gaussian_loader = DataLoader(dataset=sequences, batch_size=self.batch_size, drop_last=True,
                                           sampler=SubsetRandomSampler(indices[-split_point:]), pin_memory=True)

        self.lstmed = LSTMEDModule(X.shape[1], self.hidden_size,
                                   self.n_layers, self.use_bias, self.dropout,
                                   seed=self.seed, gpu=self.gpu)
        self.to_device(self.lstmed)
        optimizer = torch.optim.Adam(self.lstmed.parameters(), lr=self.lr)

        self.lstmed.train()
        for epoch in trange(self.num_epochs):
            logging.debug(f'Epoch {epoch+1}/{self.num_epochs}.')
            for ts_batch in train_loader:
                output = self.lstmed(self.to_var(ts_batch))
                loss = nn.MSELoss(size_average=False)(output, self.to_var(ts_batch.float()))
                self.lstmed.zero_grad()
                loss.backward()
                optimizer.step()

        self.lstmed.eval()
        error_vectors = []
        for ts_batch in train_gaussian_loader:
            output = self.lstmed(self.to_var(ts_batch))
            error = nn.L1Loss(reduce=False)(output, self.to_var(ts_batch.float()))
            error_vectors += list(error.view(-1, X.shape[1]).data.cpu().numpy())

        self.mean = np.mean(error_vectors, axis=0)
        self.cov = np.cov(error_vectors, rowvar=False)

    def predict(self, X: pd.DataFrame):
        X.interpolate(inplace=True)
        X.bfill(inplace=True)
        data = X.values
        sequences = [data[i:i + self.sequence_length] for i in range(data.shape[0] - self.sequence_length + 1)]
        data_loader = DataLoader(dataset=sequences, batch_size=self.batch_size, shuffle=False, drop_last=False)

        self.lstmed.eval()
        mvnormal = multivariate_normal(self.mean, self.cov, allow_singular=True)
        scores = []
        outputs = []
        errors = []
        for idx, ts in enumerate(data_loader):
            output = self.lstmed(self.to_var(ts))
            error = nn.L1Loss(reduce=False)(output, self.to_var(ts.float()))
            score = -mvnormal.logpdf(error.view(-1, X.shape[1]).data.cpu().numpy())
            scores.append(score.reshape(ts.size(0), self.sequence_length))
            if self.details:
                outputs.append(output.data.numpy())
                errors.append(error.data.numpy())

        # stores seq_len-many scores per timestamp and averages them
        scores = np.concatenate(scores)
        lattice = np.full((self.sequence_length, data.shape[0]), np.nan)
        for i, score in enumerate(scores):
            lattice[i % self.sequence_length, i:i + self.sequence_length] = score
        scores = np.nanmean(lattice, axis=0)

        if self.details:
            outputs = np.concatenate(outputs)
            lattice = np.full((self.sequence_length, X.shape[0], X.shape[1]), np.nan)
            for i, output in enumerate(outputs):
                lattice[i % self.sequence_length, i:i + self.sequence_length, :] = output
            self.prediction_details.update({'reconstructions_mean': np.nanmean(lattice, axis=0).T})

            errors = np.concatenate(errors)
            lattice = np.full((self.sequence_length, X.shape[0], X.shape[1]), np.nan)
            for i, error in enumerate(errors):
                lattice[i % self.sequence_length, i:i + self.sequence_length, :] = error
            self.prediction_details.update({'errors_mean': np.nanmean(lattice, axis=0).T})

        return scores


class LSTMEDModule(nn.Module, PyTorchUtils):
    def __init__(self, n_features: int, hidden_size: int,
                 n_layers: tuple, use_bias: tuple, dropout: tuple,
                 seed: int, gpu: int):
        super().__init__()
        PyTorchUtils.__init__(self, seed, gpu)
        self.n_features = n_features
        self.hidden_size = hidden_size

        self.n_layers = n_layers
        self.use_bias = use_bias
        self.dropout = dropout

        self.encoder = nn.LSTM(self.n_features, self.hidden_size, batch_first=True,
                               num_layers=self.n_layers[0], bias=self.use_bias[0], dropout=self.dropout[0])
        self.to_device(self.encoder)
        self.decoder = nn.LSTM(self.n_features, self.hidden_size, batch_first=True,
                               num_layers=self.n_layers[1], bias=self.use_bias[1], dropout=self.dropout[1])
        self.to_device(self.decoder)
        self.hidden2output = nn.Linear(self.hidden_size, self.n_features)
        self.to_device(self.hidden2output)

    def _init_hidden(self, batch_size):
        return (self.to_var(torch.Tensor(self.n_layers[0], batch_size, self.hidden_size).zero_()),
                self.to_var(torch.Tensor(self.n_layers[0], batch_size, self.hidden_size).zero_()))

    def forward(self, ts_batch, return_latent: bool=False):
        batch_size = ts_batch.shape[0]

        # 1. Encode the timeseries to make use of the last hidden state.
        enc_hidden = self._init_hidden(batch_size)  # initialization with zero
        _, enc_hidden = self.encoder(ts_batch.float(), enc_hidden)  # .float() here or .double() for the model

        # 2. Use hidden state as initialization for our Decoder-LSTM
        dec_hidden = enc_hidden

        # 3. Also, use this hidden state to get the first output aka the last point of the reconstructed timeseries
        # 4. Reconstruct timeseries backwards
        #    * Use true data for training decoder
        #    * Use hidden2output for prediction
        output = self.to_var(torch.Tensor(ts_batch.size()).zero_())
        for i in reversed(range(ts_batch.shape[1])):
            output[:, i, :] = self.hidden2output(dec_hidden[0][0, :])

            if self.training:
                _, dec_hidden = self.decoder(ts_batch[:, i].unsqueeze(1).float(), dec_hidden)
            else:
                _, dec_hidden = self.decoder(output[:, i].unsqueeze(1), dec_hidden)

        return (output, enc_hidden[1][-1]) if return_latent else output

"""Adapted from Daniel Stanley Tan (https://github.com/danieltan07/dagmm)"""
import logging
import sys
sys.path.append("../DeepADoTS/src/algorithms/")

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from tqdm import trange

# from .algorithm_utils import Algorithm, PyTorchUtils
# from .autoencoder import AutoEncoderModule
#from lstm_enc_dec_axl import LSTMEDModule


class DAGMM(Algorithm, PyTorchUtils):
    class AutoEncoder:
        NN = AutoEncoderModule
        LSTM = LSTMEDModule

    def __init__(self, num_epochs=10, lambda_energy=0.1, lambda_cov_diag=0.005, lr=1e-3, batch_size=50, gmm_k=3,
                 normal_percentile=80, sequence_length=30, autoencoder_type=AutoEncoderModule, autoencoder_args=None,
                 hidden_size: int=5, seed: int=None, gpu: int=None, details=True):
        _name = 'LSTM-DAGMM' if autoencoder_type == LSTMEDModule else 'DAGMM'
        Algorithm.__init__(self, __name__, _name, seed, details=details)
        PyTorchUtils.__init__(self, seed, gpu)
        self.num_epochs = num_epochs
        self.lambda_energy = lambda_energy
        self.lambda_cov_diag = lambda_cov_diag
        self.lr = lr
        self.batch_size = batch_size
        self.sequence_length = sequence_length
        self.gmm_k = gmm_k  # Number of Gaussian mixtures
        self.normal_percentile = normal_percentile  # Up to which percentile data should be considered normal
        self.autoencoder_type = autoencoder_type
        if autoencoder_type == AutoEncoderModule:
            self.autoencoder_args = ({'sequence_length': self.sequence_length})
        elif autoencoder_type == LSTMEDModule:
            self.autoencoder_args = ({'n_layers': (1, 1), 'use_bias': (True, True), 'dropout': (0.0, 0.0)})
        self.autoencoder_args.update({'seed': seed, 'gpu': gpu})
        if autoencoder_args is not None:
            self.autoencoder_args.update(autoencoder_args)
        self.hidden_size = hidden_size

        self.dagmm, self.optimizer, self.train_energy, self._threshold = None, None, None, None

    def reset_grad(self):
        self.dagmm.zero_grad()

    def dagmm_step(self, input_data):
        print ("input_data",input_data)
        self.dagmm.train()
        enc, dec, z, gamma = self.dagmm(input_data)
        #print (enc, dec, z, gamma)
        total_loss, sample_energy, recon_error, cov_diag = self.dagmm.loss_function(input_data, dec, z, gamma,
                                                                                    self.lambda_energy,
                                                                                    self.lambda_cov_diag)
        print ("total_loss",total_loss)
        self.reset_grad()
        total_loss = torch.clamp(total_loss, max=1e7)  # Extremely high loss can cause NaN gradients
        #total_loss = total_loss.float() 
        #total_loss = total_loss.cuda()
        #total_loss = total_loss.cpu()
        
        #total_loss=total_loss.type(torch.FloatTensor)
        #total_loss = total_loss.to("cuda:0")
        print ("total_loss", total_loss)
        total_loss.backward()
        torch.nn.utils.clip_grad_norm_(self.dagmm.parameters(), 5)
        # if np.array([np.isnan(p.grad.detach().numpy()).any() for p in self.dagmm.parameters()]).any():
        #     import IPython; IPython.embed()
        self.optimizer.step()
        return total_loss, sample_energy, recon_error, cov_diag

    def fit(self, X: pd.DataFrame):
        """Learn the mixture probability, mean and covariance for each component k.
        Store the computed energy based on the training data and the aforementioned parameters."""
        #X.interpolate(inplace=True)
        X.bfill(inplace=True)
        data = X.values
        sequences = [data[i:i + self.sequence_length] for i in range(X.shape[0] - self.sequence_length + 1)]
        data_loader = DataLoader(dataset=sequences, batch_size=self.batch_size, shuffle=True, drop_last=True)
        self.hidden_size = 5 + int(X.shape[1] / 20)
        autoencoder = self.autoencoder_type(X.shape[1], hidden_size=self.hidden_size, **self.autoencoder_args)
        self.dagmm = DAGMMModule(autoencoder, n_gmm=self.gmm_k, latent_dim=self.hidden_size + 2,
                                 seed=self.seed, gpu=self.gpu)
        self.to_device(self.dagmm)
        self.optimizer = torch.optim.Adam(self.dagmm.parameters(), lr=self.lr)

        for _ in trange(self.num_epochs):
            for input_data in data_loader:
                input_data = self.to_var(input_data)
                self.dagmm_step(input_data.float())

        self.dagmm.eval()
        n = 0
        mu_sum = 0
        cov_sum = 0
        gamma_sum = 0
        for input_data in data_loader:
            input_data = self.to_var(input_data)
            _, _, z, gamma = self.dagmm(input_data.float())
            phi, mu, cov = self.dagmm.compute_gmm_params(z, gamma)

            batch_gamma_sum = torch.sum(gamma, dim=0)

            gamma_sum += batch_gamma_sum
            mu_sum += mu * batch_gamma_sum.unsqueeze(-1)  # keep sums of the numerator only
            cov_sum += cov * batch_gamma_sum.unsqueeze(-1).unsqueeze(-1)  # keep sums of the numerator only

            n += input_data.size(0)

    def predict(self, X: pd.DataFrame):
        """Using the learned mixture probability, mean and covariance for each component k, compute the energy on the
        given data."""
        self.dagmm.eval()
        #X.interpolate(inplace=True)
        X.bfill(inplace=True)
        data = X.values
        sequences = [data[i:i + self.sequence_length] for i in range(len(data) - self.sequence_length + 1)]
        data_loader = DataLoader(dataset=sequences, batch_size=1, shuffle=False)
        test_energy = np.full((self.sequence_length, X.shape[0]), np.nan)

        encodings = np.full((self.sequence_length, X.shape[0], self.hidden_size), np.nan)
        decodings = np.full((self.sequence_length, X.shape[0], X.shape[1]), np.nan)
        euc_errors = np.full((self.sequence_length, X.shape[0]), np.nan)
        csn_errors = np.full((self.sequence_length, X.shape[0]), np.nan)

        for i, sequence in enumerate(data_loader):
            #print ("shape of sequence",self.to_var(sequence).float().shape)
            enc, dec, z, gamma = self.dagmm(self.to_var(sequence).float())
            sample_energy, _ = self.dagmm.compute_energy(z, size_average=False)
            idx = (i % self.sequence_length, np.arange(i, i + self.sequence_length))
            test_energy[idx] = sample_energy.data.numpy()

            if self.details:
                encodings[idx] = enc.data.cpu().numpy()
                decodings[idx] = dec.data.cpu().numpy()
                euc_errors[idx] = z[:, 1].data.cpu().numpy()
                csn_errors[idx] = z[:, 2].data.cpu().numpy()

        test_energy = np.nanmean(test_energy, axis=0)

        if self.details:
            self.prediction_details.update({'latent_representations': np.nanmean(encodings, axis=0).T})
            self.prediction_details.update({'reconstructions_mean': np.nanmean(decodings, axis=0).T})
            self.prediction_details.update({'euclidean_errors_mean': np.nanmean(euc_errors, axis=0)})
            self.prediction_details.update({'cosine_errors_mean': np.nanmean(csn_errors, axis=0)})

        return test_energy


# class DAGMMModule(nn.Module, PyTorchUtils):
#     """Residual Block."""

#     def __init__(self, autoencoder, n_gmm, latent_dim, seed: int, gpu: int):
#         super(DAGMMModule, self).__init__()
#         PyTorchUtils.__init__(self, seed, gpu)

#         self.add_module('autoencoder', autoencoder)

#         layers = [
#             nn.Linear(latent_dim, 10),
#             nn.Tanh(),
#             nn.Dropout(p=0.5),
#             nn.Linear(10, n_gmm),
#             nn.Softmax(dim=1)
#         ]
#         self.estimation = nn.Sequential(*layers)
#         self.to_device(self.estimation)

#         self.register_buffer('phi', self.to_var(torch.zeros(n_gmm)))
#         self.register_buffer('mu', self.to_var(torch.zeros(n_gmm, latent_dim)))
#         self.register_buffer('cov', self.to_var(torch.zeros(n_gmm, latent_dim, latent_dim)))

#     def relative_euclidean_distance(self, a, b, dim=1):
#         return (a - b).norm(2, dim=dim) / torch.clamp(a.norm(2, dim=dim), min=1e-10)

#     def forward(self, x):
#         dec, enc = self.autoencoder(x, return_latent=True)

#         rec_cosine = F.cosine_similarity(x.view(x.shape[0], -1), dec.view(dec.shape[0], -1), dim=1)
#         rec_euclidean = self.relative_euclidean_distance(x.view(x.shape[0], -1), dec.view(dec.shape[0], -1), dim=1)

#         # Concatenate latent representation, cosine similarity and relative Euclidean distance between x and dec(enc(x))
#         z = torch.cat([enc, rec_euclidean.unsqueeze(-1), rec_cosine.unsqueeze(-1)], dim=1)
#         gamma = self.estimation(z)

#         return enc, dec, z, gamma

#     def compute_gmm_params(self, z, gamma):
#         N = gamma.size(0)
#         # K
#         sum_gamma = torch.sum(gamma, dim=0)

#         # K
#         phi = (sum_gamma / N)

#         self.phi = phi.data

#         # K x D
#         mu = torch.sum(gamma.unsqueeze(-1) * z.unsqueeze(1), dim=0) / sum_gamma.unsqueeze(-1)
#         self.mu = mu.data
#         # z = N x D
#         # mu = K x D
#         # gamma N x K

#         # z_mu = N x K x D
#         z_mu = (z.unsqueeze(1) - mu.unsqueeze(0))

#         # z_mu_outer = N x K x D x D
#         z_mu_outer = z_mu.unsqueeze(-1) * z_mu.unsqueeze(-2)

#         # K x D x D
#         cov = torch.sum(gamma.unsqueeze(-1).unsqueeze(-1) * z_mu_outer, dim=0) / sum_gamma.unsqueeze(-1).unsqueeze(-1)
#         self.cov = cov.data

#         return phi, mu, cov

#     def compute_energy(self, z, phi=None, mu=None, cov=None, size_average=True):
#         if phi is None:
#             phi = Variable(self.phi)
#         if mu is None:
#             mu = Variable(self.mu)
#         if cov is None:
#             cov = Variable(self.cov)

#         k, d, _ = cov.size()

#         z_mu = (z.unsqueeze(1) - mu.unsqueeze(0))

#         cov_inverse = []
#         det_cov = []
#         cov_diag = 0
#         eps = 1e-12
#         for i in range(k):
#             # K x D x D
#             cov_k = cov[i] + self.to_var(torch.eye(d) * eps)
#             pinv = np.linalg.pinv(cov_k.data.cpu().numpy())
#             cov_inverse.append(Variable(torch.from_numpy(pinv)).unsqueeze(0))

#             eigvals = np.linalg.eigvals(cov_k.data.cpu().numpy() * (2 * np.pi))
#             if np.min(eigvals) < 0:
#                 pass
#                 #logging.warning(f'Determinant was negative! Clipping Eigenvalues to 0+epsilon from {np.min(eigvals)}')
#             determinant = np.prod(np.clip(eigvals, a_min=sys.float_info.epsilon, a_max=None))
#             det_cov.append(determinant)

#             cov_diag = cov_diag + torch.sum(1 / cov_k.diag())

#         # K x D x D
#         cov_inverse = torch.cat(cov_inverse, dim=0)
#         # K
#         det_cov = Variable(torch.from_numpy(np.float32(np.array(det_cov))))
# #         print ("sum-0",cov_inverse.unsqueeze(0))
# #         print ("sum-1",z_mu.ufnsqueeze(-1).cpu())
# #         print ("sum", torch.sum(z_mu.unsqueeze(-1).cpu() * cov_inverse.unsqueeze(0), dim=-2))
#         # N x K
#         exp_term_tmp = -0.5 * torch.sum(torch.sum(z_mu.unsqueeze(-1).cpu() * cov_inverse.unsqueeze(0), dim=-2) * z_mu.cpu(), dim=-1)
#         # for stability (logsumexp)
#         max_val = torch.max((exp_term_tmp).clamp(min=0), dim=1, keepdim=True)[0]

#         exp_term = torch.exp(exp_term_tmp - max_val)
# #         print ("sample_energy", self.to_var(phi.unsqueeze(0)).cpu())
# #         print ("sample_energy-exp_term", exp_term)
#         sample_energy = -max_val.squeeze() - torch.log(
#             torch.sum(self.to_var(phi.unsqueeze(0)).cpu() * exp_term / (torch.sqrt(self.to_var(det_cov).cpu()) + eps).unsqueeze(0),
#                       dim=1) + eps)

#         if size_average:
#             sample_energy = torch.mean(sample_energy)

#         return sample_energy, cov_diag

#     def loss_function(self, x, x_hat, z, gamma, lambda_energy, lambda_cov_diag):
#         recon_error = torch.mean((x.view(*x_hat.shape) - x_hat) ** 2)
#         #print (z, gamma)
#         phi, mu, cov = self.compute_gmm_params(z, gamma)
        
#         #print (z, phi, mu, cov)
#         sample_energy, cov_diag = self.compute_energy(z, phi, mu, cov)
# #         print ("recon_error",recon_error)
# #         print ("lambda_energy",lambda_energy)
# #         print ("lambda_cov_diag",lambda_cov_diag)
# #         cov_diag = cov_diag.float()
# #         print ("cov_diag",cov_diag)
#         loss = recon_error + lambda_energy * sample_energy + lambda_cov_diag * cov_diag
        
#         return loss, sample_energy, recon_error, cov_diag

In [130]:
class Algorithm(metaclass=abc.ABCMeta):
    def __init__(self, module_name, name, framework):
        self.logger = logging.getLogger(module_name)
        self.name = name
        self.framework = framework

    def __str__(self) -> str:
        return self.name

    @abc.abstractmethod
    def fit(self, X, y):
        """
        Train the algorithm on the given dataset
        :param X:
        :param y:
        :return: self
        """

    @abc.abstractmethod
    def predict(self, X):
        """
        :return score
        """

    @abc.abstractmethod
    def binarize(self, score, threshold=None):
        """
        :param threshold:
        :param score
        :return binary_labels
        """

    @abc.abstractmethod
    def threshold(self, score):
        """
        :param score
        :return threshold:
        """

    class Frameworks:
        PyTorch, Tensorflow = range(2)
        
class NNAutoEncoder(AutoEncoder, GPUWrapper):
    def __init__(self, n_features=118, sequence_length=1, hidden_size=1, gpu=0):
        AutoEncoder.__init__(self)
        GPUWrapper.__init__(self, gpu)
        n_features = n_features * sequence_length

        layers = []
        layers += [nn.Linear(n_features, 60)]
        layers += [nn.Tanh()]
        layers += [nn.Linear(60, 30)]
        layers += [nn.Tanh()]
        layers += [nn.Linear(30, 10)]
        layers += [nn.Tanh()]
        layers += [nn.Linear(10, hidden_size)]

        self._encoder = nn.Sequential(*layers)
        self.to_device(self._encoder)

        layers = []
        layers += [nn.Linear(hidden_size, 10)]
        layers += [nn.Tanh()]
        layers += [nn.Linear(10, 30)]
        layers += [nn.Tanh()]
        layers += [nn.Linear(30, 60)]
        layers += [nn.Tanh()]
        layers += [nn.Linear(60, n_features)]

        self._decoder = nn.Sequential(*layers)
        self.to_device(self._decoder)

    def forward(self, x):
        x = x.view(x.shape[0], -1)

        enc = self._encoder(x)
        dec = self._decoder(enc)

        return dec, enc


class LSTMAutoEncoder(AutoEncoder, GPUWrapper):
    """Autoencoder with Recurrent module. Inspired by LSTM-EncDec"""

    def __init__(self, n_features: int, sequence_length: int, hidden_size: int = 1, n_layers: tuple = (3, 3),
                 use_bias: tuple = (True, True), dropout: tuple = (0.3, 0.3), gpu: int = 0):
        AutoEncoder.__init__(self)
        GPUWrapper.__init__(self, gpu)

        self.n_features = n_features
        self.sequence_length = sequence_length
        self.hidden_size = hidden_size

        self.n_layers = n_layers
        self.use_bias = use_bias
        self.dropout = dropout

        self.encoder = nn.LSTM(self.n_features, self.hidden_size, batch_first=True,
                               num_layers=self.n_layers[0], bias=self.use_bias[0], dropout=self.dropout[0])
        self.to_device(self.encoder)
        self.decoder = nn.LSTM(self.n_features, self.hidden_size, batch_first=True,
                               num_layers=self.n_layers[1], bias=self.use_bias[1], dropout=self.dropout[1])
        self.to_device(self.decoder)
        self.hidden2output = nn.Linear(self.hidden_size, self.n_features)
        self.to_device(self.hidden2output)

    def _init_hidden(self, batch_size):
        return (self.to_var(torch.zeros(self.n_layers[0], batch_size, self.hidden_size)),
                self.to_var(torch.zeros(self.n_layers[0], batch_size, self.hidden_size)))

    def forward(self, ts_batch):
        batch_size = ts_batch.shape[0]

        # 1. Encode the timeseries to make use of the last hidden state.
        enc_hidden = self._init_hidden(ts_batch.shape[0])  # initialization with zero
        _, enc_hidden = self.encoder(ts_batch.float(), enc_hidden)  # .float() here or .double() for the model

        # 2. Use hidden state as initialization for our Decoder-LSTM
        dec_hidden = (enc_hidden[0], self.to_var(torch.zeros(self.n_layers[1], batch_size, self.hidden_size)))

        # 3. Also, use this hidden state to get the first output aka the last point of the reconstructed timeseries
        # 4. Reconstruct timeseries backwards
        #    * Use true data for training decoder
        #    * Use hidden2output for prediction
        output = self.to_var(torch.zeros(ts_batch.size()))
        for i in reversed(range(ts_batch.shape[1])):
            output[:, i, :] = self.hidden2output(dec_hidden[0][0, :])

            if self.training:
                _, dec_hidden = self.decoder(ts_batch[:, i].unsqueeze(1).float(), dec_hidden)
            else:
                _, dec_hidden = self.decoder(output[:, i].unsqueeze(1), dec_hidden)

        return output.squeeze(2), enc_hidden[0][-1].view(batch_size, self.hidden_size)
class DAGMMModule(nn.Module, GPUWrapper):
    """Residual Block."""

    def __init__(self, autoencoder, n_gmm=2, latent_dim=3, gpu=0):
        super(DAGMMModule, self).__init__()
        GPUWrapper.__init__(self, gpu)

        self.add_module('autoencoder', autoencoder)

        layers = [
            nn.Linear(latent_dim, 10),
            nn.Tanh(),
            nn.Dropout(p=0.5),
            nn.Linear(10, n_gmm),
            nn.Softmax(dim=1)
        ]
        self.estimation = nn.Sequential(*layers)
        self.to_device(self.estimation)

        self.register_buffer('phi', self.to_var(torch.zeros(n_gmm)))
        self.register_buffer('mu', self.to_var(torch.zeros(n_gmm, latent_dim)))
        self.register_buffer('cov', self.to_var(torch.zeros(n_gmm, latent_dim, latent_dim)))

    def relative_euclidean_distance(self, a, b, dim=1):
        return (a - b).norm(2, dim=dim) / torch.clamp(a.norm(2, dim=dim), min=1e-10)

    def forward(self, x):
        dec, enc = self.autoencoder(x)

        rec_cosine = F.cosine_similarity(x.view(x.shape[0], -1), dec.view(dec.shape[0], -1), dim=1)
        rec_euclidean = self.relative_euclidean_distance(x.view(x.shape[0], -1), dec.view(dec.shape[0], -1), dim=1)

        # Concatenate latent representation, cosine similarity and relative Euclidean distance between x and dec(enc(x))
        z = torch.cat([enc, rec_euclidean.unsqueeze(-1), rec_cosine.unsqueeze(-1)], dim=1)
        gamma = self.estimation(z)

        return enc, dec, z, gamma

    def compute_gmm_params(self, z, gamma):
        N = gamma.size(0)
        # K
        sum_gamma = torch.sum(gamma, dim=0)

        # K
        phi = (sum_gamma / N)

        self.phi = phi.data

        # K x D
        mu = torch.sum(gamma.unsqueeze(-1) * z.unsqueeze(1), dim=0) / sum_gamma.unsqueeze(-1)
        self.mu = mu.data
        # z = N x D
        # mu = K x D
        # gamma N x K

        # z_mu = N x K x D
        z_mu = (z.unsqueeze(1) - mu.unsqueeze(0))

        # z_mu_outer = N x K x D x D
        z_mu_outer = z_mu.unsqueeze(-1) * z_mu.unsqueeze(-2)

        # K x D x D
        cov = torch.sum(gamma.unsqueeze(-1).unsqueeze(-1) * z_mu_outer, dim=0) / sum_gamma.unsqueeze(-1).unsqueeze(-1)
        self.cov = cov.data

        return phi, mu, cov

    def compute_energy(self, z, phi=None, mu=None, cov=None, size_average=True):
        if phi is None:
            phi = Variable(self.phi)
        if mu is None:
            mu = Variable(self.mu)
        if cov is None:
            cov = Variable(self.cov)

        k, d, _ = cov.size()

        z_mu = (z.unsqueeze(1) - mu.unsqueeze(0))

        cov_inverse = []
        det_cov = []
        cov_diag = 0
        eps = 1e-12
        for i in range(k):
            # K x D x D
            cov_k = cov[i] + self.to_var(torch.eye(d) * eps)
            cov_inverse.append(torch.inverse(cov_k).unsqueeze(0))

            eigvals = np.linalg.eigvals(cov_k.data.cpu().numpy() * (2 * np.pi))
            if np.min(eigvals) < 0:
                logging.warning(f'Determinant was negative! Clipping Eigenvalues to 0+epsilon from {np.min(eigvals)}')
            determinant = np.prod(np.clip(eigvals, a_min=sys.float_info.epsilon, a_max=None))
            det_cov.append(determinant)

            cov_diag = cov_diag + torch.sum(1 / cov_k.diag())

        # K x D x D
        cov_inverse = torch.cat(cov_inverse, dim=0)
        # K
        det_cov = Variable(torch.from_numpy(np.float32(np.array(det_cov))))

        # N x K
        exp_term_tmp = -0.5 * torch.sum(torch.sum(z_mu.unsqueeze(-1) * cov_inverse.unsqueeze(0), dim=-2) * z_mu, dim=-1)
        # for stability (logsumexp)
        max_val = torch.max((exp_term_tmp).clamp(min=0), dim=1, keepdim=True)[0]

        exp_term = torch.exp(exp_term_tmp - max_val)

        sample_energy = -max_val.squeeze() - torch.log(
            torch.sum(self.to_var(phi.unsqueeze(0)) * exp_term / (torch.sqrt(self.to_var(det_cov))).unsqueeze(0),
                      dim=1) + eps)

        if size_average:
            sample_energy = torch.mean(sample_energy)

        return sample_energy, cov_diag

    def loss_function(self, x, x_hat, z, gamma, lambda_energy, lambda_cov_diag):
        recon_error = torch.mean((x.view(*x_hat.shape) - x_hat) ** 2)
        phi, mu, cov = self.compute_gmm_params(z, gamma)
        sample_energy, cov_diag = self.compute_energy(z, phi, mu, cov)
        loss = recon_error + lambda_energy * sample_energy + lambda_cov_diag * cov_diag
        return loss, sample_energy, recon_error, cov_diag


class DAGMM(Algorithm, GPUWrapper):
    def __init__(self, num_epochs=10, lambda_energy=0.1, lambda_cov_diag=0.005, lr=1e-2, batch_size=50, gmm_k=3,
                 normal_percentile=80, sequence_length=15, autoencoder_type=NNAutoEncoder, autoencoder_args=None,
                 framework=Algorithm.Frameworks.PyTorch, gpu: int=0):
        window_name = 'withWindow' if sequence_length > 1 else 'withoutWindow'
        Algorithm.__init__(self, __name__, f'DAGMM_{autoencoder_type.__name__}_{window_name}', framework)
        GPUWrapper.__init__(self, gpu)
        self.num_epochs = num_epochs
        self.lambda_energy = lambda_energy
        self.lambda_cov_diag = lambda_cov_diag
        self.lr = lr
        self.batch_size = batch_size
        self.sequence_length = sequence_length
        self.gmm_k = gmm_k  # Number of Gaussian mixtures
        self.normal_percentile = normal_percentile  # Up to which percentile data should be considered normal
        self.autoencoder_type = autoencoder_type
        self.autoencoder_args = autoencoder_args or {'gpu': gpu}

        self.dagmm, self.optimizer, self.train_energy, self._threshold = None, None, None, None

    def reset_grad(self):
        self.dagmm.zero_grad()

    def dagmm_step(self, input_data):
        self.dagmm.train()
        enc, dec, z, gamma = self.dagmm(input_data)
        total_loss, sample_energy, recon_error, cov_diag = self.dagmm.loss_function(input_data, dec, z, gamma,
                                                                                    self.lambda_energy,
                                                                                    self.lambda_cov_diag)
        self.reset_grad()
        total_loss = torch.clamp(total_loss, max=1e8)  # Extremely high loss can cause NaN gradients
        total_loss.backward()
        torch.nn.utils.clip_grad_norm_(self.dagmm.parameters(), 5)
        self.optimizer.step()
        return total_loss, sample_energy, recon_error, cov_diag

    def fit(self, X: pd.DataFrame):
        """Learn the mixture probability, mean and covariance for each component k.
        Store the computed energy based on the training data and the aforementioned parameters."""
        X = X.dropna()
        data = X.values
        # Each point is a flattened window and thus has as many features as sequence_length * features
        multi_points = [data[i:i + self.sequence_length] for i in range(len(data) - self.sequence_length + 1)]
        data_loader = DataLoader(dataset=multi_points, batch_size=self.batch_size, shuffle=True, drop_last=True)
        hidden_size = max(1, X.shape[1] // 20)
        autoencoder = self.autoencoder_type(n_features=X.shape[1], sequence_length=self.sequence_length,
                                            hidden_size=hidden_size, **self.autoencoder_args)
        self.dagmm = DAGMMModule(autoencoder, n_gmm=self.gmm_k, latent_dim=hidden_size + 2, gpu=self.gpu)
        self.to_device(self.dagmm)
        self.optimizer = torch.optim.Adam(self.dagmm.parameters(), lr=self.lr)

        for _ in range(self.num_epochs):
            for input_data in data_loader:
                input_data = self.to_var(input_data)
                self.dagmm_step(input_data.float())

        self.dagmm.eval()
        n = 0
        mu_sum = 0
        cov_sum = 0
        gamma_sum = 0
        for input_data in data_loader:
            input_data = self.to_var(input_data)
            _, _, z, gamma = self.dagmm(input_data.float())
            phi, mu, cov = self.dagmm.compute_gmm_params(z, gamma)

            batch_gamma_sum = torch.sum(gamma, dim=0)

            gamma_sum += batch_gamma_sum
            mu_sum += mu * batch_gamma_sum.unsqueeze(-1)  # keep sums of the numerator only
            cov_sum += cov * batch_gamma_sum.unsqueeze(-1).unsqueeze(-1)  # keep sums of the numerator only

            n += input_data.size(0)

        train_phi = gamma_sum / n
        train_mu = mu_sum / gamma_sum.unsqueeze(-1)
        train_cov = cov_sum / gamma_sum.unsqueeze(-1).unsqueeze(-1)

        train_length = len(data_loader) * self.batch_size + self.sequence_length - 1
        train_energy = np.full((self.sequence_length, train_length), np.nan)
        for i1, ts_batch in enumerate(data_loader):
            ts_batch = self.to_var(ts_batch)
            _, _, z, _ = self.dagmm(ts_batch.float())
            sample_energies, _ = self.dagmm.compute_energy(z, phi=train_phi, mu=train_mu, cov=train_cov,
                                                           size_average=False)

            for i2, sample_energy in enumerate(sample_energies):
                index = i1 * self.batch_size + i2
                window_elements = list(range(index, index + self.sequence_length, 1))
                train_energy[index % self.sequence_length, window_elements] = sample_energy.data.cpu().numpy()
        self.train_energy = np.nanmedian(train_energy, axis=0)

    def predict(self, X: pd.DataFrame):
        """Using the learned mixture probability, mean and covariance for each component k, compute the energy on the
        given data."""
        self.dagmm.eval()
        X = X.dropna()
        data = X.values
        multi_points = [data[i:i + self.sequence_length] for i in range(len(data) - self.sequence_length + 1)]
        data_loader = DataLoader(dataset=multi_points, batch_size=1, shuffle=False)
        test_energy = np.full((self.sequence_length, len(data)), np.nan)

        for idx, multi_point in enumerate(data_loader):
            _, _, z, _ = self.dagmm(self.to_var(multi_point).float())
            sample_energy, _ = self.dagmm.compute_energy(z, size_average=False)
            window_elements = np.arange(idx, idx + self.sequence_length, 1)
            test_energy[idx % self.sequence_length, window_elements] = sample_energy.data.cpu().numpy()

        test_energy = np.nanmedian(test_energy, axis=0)
        combined_energy = np.concatenate([self.train_energy, test_energy], axis=0)

        self._threshold = np.nanpercentile(combined_energy, self.normal_percentile)
        return test_energy

    def threshold(self, score):
        return self._threshold

    def binarize(self, y, threshold=None):
        if threshold is None:
            if self._threshold is not None:
                threshold = self._threshold
            else:
                return np.zeros_like(y)
        return np.where(y > threshold, 1, 0)

In [131]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"]="0"  
detectors = DAGMM(num_epochs=100, sequence_length=3, gpu = 0)

In [132]:
help(detectors)

Help on DAGMM in module __main__ object:

class DAGMM(Algorithm, GPUWrapper)
 |  Method resolution order:
 |      DAGMM
 |      Algorithm
 |      GPUWrapper
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, num_epochs=10, lambda_energy=0.1, lambda_cov_diag=0.005, lr=0.01, batch_size=50, gmm_k=3, normal_percentile=80, sequence_length=15, autoencoder_type=<class '__main__.NNAutoEncoder'>, autoencoder_args=None, framework=0, gpu:int=0)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  binarize(self, y, threshold=None)
 |      :param threshold:
 |      :param score
 |      :return binary_labels
 |  
 |  dagmm_step(self, input_data)
 |  
 |  fit(self, X:pandas.core.frame.DataFrame)
 |      Learn the mixture probability, mean and covariance for each component k.
 |      Store the computed energy based on the training data and the aforementioned parameters.
 |  
 |  predict(self, X:pandas.core.frame.DataFrame)
 |      Using the learne

In [133]:
detectors.fit(X_train.iloc[:52,:-1].copy())


TypeError: __init__() got an unexpected keyword argument 'details'

In [115]:
X_train.shape

(4171146, 24)

# output features and save it

In [ ]:
score = detectors.predict(Feature.iloc[:,:-1].copy())
output = pd.DataFrame({"cano_locdt_index":Feature.iloc[:,-1]})
output["score"] = score
print (output.shape)

In [ ]:
output["cosine_errors_mean"] = detectors.prediction_details["cosine_errors_mean"]
output["euclidean_errors_mean"]  = detectors.prediction_details["euclidean_errors_mean"]
data = detectors.prediction_details["reconstructions_mean"]
reconstructions_mean = pd.DataFrame(data.T,
             columns = ["reconstructions_mean_latent_features_{}".format(i) for i in range(data.shape[0])]
            )
print (reconstructions_mean.shape)
data = detectors.prediction_details["latent_representations"]
latent_representations = pd.DataFrame(data.T,
             columns = ["latent_representations_latent_features_{}".format(i) for i in range(data.shape[0])]
            )
print (latent_representations.shape)
output = pd.concat([output,reconstructions_mean,latent_representations], axis = 1)
print (output.shape)

In [ ]:
feature = []
for i in range(len(output)):
    if i%3 == 2:
        feature.append(output.iloc[i:i+1])
feature = pd.concat(feature,axis = 0)

In [ ]:
feature

In [ ]:
feature.to_csv("/data/yunrui_li/fraud/fraud_detection/features/DAGMM_features.csv", index = False)

In [ ]:
assert 1==0

In [ ]:
gpu = 0
torch.device(f'cuda:{gpu}' if torch.cuda.is_available() and gpu is not None else 'cpu')

In [ ]:
import torch
os.environ["CUDA_VISIBLE_DEVICES"]="0"  

torch.cuda.current_device()

In [ ]:
torch.cuda.get_device_name(0)

In [ ]:
from tensorflow.python.client import device_lib
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"  
local_device_protos = device_lib.list_local_devices()
gpus = [x.name for x in local_device_protos if x.device_type == 'GPU']


In [ ]:
gpus

In [ ]:
local_device_protos